In [55]:
from google.colab import drive
drive.mount('/content/drive')

BASE_DIR = "/content/drive/MyDrive/sorbet_data"   # 你的数据目录
SOURCE_OWL = "saref4bldg.rdf"                         # 作为 source 的本体文件名（可改）
TARGET_OWL = "Sargon.owl"                     # 作为 target 的本体文件名（可改）

import os, pathlib
assert os.path.exists(os.path.join(BASE_DIR, SOURCE_OWL)), f"缺少 {SOURCE_OWL}"
assert os.path.exists(os.path.join(BASE_DIR, TARGET_OWL)), f"缺少 {TARGET_OWL}"
pathlib.Path(f"{BASE_DIR}/refs_equiv").mkdir(parents=True, exist_ok=True)

RESULTS_DIR = "/content/drive/MyDrive/sorbet_data/result_alignments"  # 结果直接写入Drive
pathlib.Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)
print("✔ 数据&结果目录OK")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✔ 数据&结果目录OK


In [56]:
%cd /content
!rm -rf SORBET_ISWC23
!git clone https://github.com/Lama-West/SORBET_ISWC23.git
%cd /content/SORBET_ISWC23
!ls -la | head -n 20


/content
Cloning into 'SORBET_ISWC23'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 97 (delta 33), reused 94 (delta 30), pack-reused 0 (from 0)
Receiving objects: 100% (97/97), 222.54 KiB | 22.25 MiB/s, done.
Resolving deltas: 100% (33/33), done.
/content/SORBET_ISWC23
total 56
drwxr-xr-x 5 root root  4096 Oct 10 21:41 .
drwxr-xr-x 1 root root  4096 Oct 10 21:41 ..
-rw-r--r-- 1 root root  7019 Oct 10 21:41 environment.yml
drwxr-xr-x 2 root root  4096 Oct 10 21:41 figures
drwxr-xr-x 8 root root  4096 Oct 10 21:41 .git
-rw-r--r-- 1 root root   812 Oct 10 21:41 .gitignore
-rw-r--r-- 1 root root  8549 Oct 10 21:41 README.md
-rw-r--r-- 1 root root 10158 Oct 10 21:41 requirements.txt
drwxr-xr-x 7 root root  4096 Oct 10 21:41 src


In [57]:
# 先卸掉常见冲突包
!python -m pip uninstall -y -q opencv-python opencv-python-headless opencv-contrib-python tsfresh thinc arviz mizani plotnine umap-learn imbalanced-learn dopamine-rl

# 安装核心依赖（与本仓库配合良好的一组版本）
!python -m pip install -q --no-cache-dir \
  "numpy==1.26.4" "scipy==1.13.1" "pandas==2.2.2" "scikit-learn==1.4.2" \
  "w3lib==2.2.1" "autocorrect==2.6.1" "nltk==3.9.1" "owlready2==0.46" \
  "rdflib==7.0.0" "gensim==4.3.2" "tqdm==4.66.5" "networkx==3.2.1"

# 下载 NLTK 词表
import nltk
nltk.download('stopwords')
print("✔ 依赖安装完成")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.3/23.3 MB 312.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 178.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 216.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 189.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.8.7 requires thinc<8.4.0,>=8.3.4, which is not installed.
dataproc-spark-connect 0.8.3 requires tqdm>=4.67, but you have tqdm 4.66.5 which is incompatible.
cuml-cu12 25.6.0 requires scikit-learn>=1.5, but you have scikit-learn 1.4.2 which is incompatible.
✔ 依赖安装完成


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [58]:
import json, os

cfg = {
  "General": { "model": "sentence-transformers/all-MiniLM-L6-v2" },
  "energy_pair": {
    "ontologies_folder": BASE_DIR,
    "ontologies": [SOURCE_OWL, TARGET_OWL],
    "alignments_folder": f"{BASE_DIR}/refs_equiv",
    "ontologies_in_alignment": [
        os.path.splitext(SOURCE_OWL)[0],
        os.path.splitext(TARGET_OWL)[0]
    ],
    "alignments": [],          # 无训练标注
    "test_alignments": "",     # 先留空，下一格生成最小金标后再指向
    "parsing_parameters": {
      "use_synonyms": 1,
      "exclude_classes": [],
      "subclass_of_properties": ["rdfs:subClassOf"],
      "autocorrect": 0,
      "synonym_extension": 0
    }
  }
}
with open('config.json','w',encoding='utf-8') as f:
    json.dump(cfg, f, indent=2, ensure_ascii=False)
print("✔ config.json 已写好")


✔ config.json 已写好


In [59]:
from rdflib import Graph, RDF, OWL
import os

left  = os.path.join(BASE_DIR, SOURCE_OWL)
right = os.path.join(BASE_DIR, TARGET_OWL)
refs  = os.path.join(BASE_DIR, "refs_equiv")
os.makedirs(refs, exist_ok=True)

def pick_class_iri(path):
    g = Graph(); g.parse(path)
    for s in g.subjects(RDF.type, OWL.Class):
        return str(s)
    return str(OWL.Thing)

src_iri = pick_class_iri(left)   # source 的任意一个类
tgt_iri = pick_class_iri(right)  # target 的任意一个类

xml = f'''<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF xmlns="http://knowledgeweb.semanticweb.org/heterogeneity/alignment#"
         xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
  <Alignment>
    <xml>yes</xml><level>0</level><type>**</type>
    <onto1><Ontology rdf:about="file://{left}"/></onto1>
    <onto2><Ontology rdf:about="file://{right}"/></onto2>
    <map>
      <Cell>
        <entity1 rdf:resource="{src_iri}"/>
        <entity2 rdf:resource="{tgt_iri}"/>
        <relation>=</relation><measure>1.0</measure>
      </Cell>
    </map>
  </Alignment>
</rdf:RDF>
'''
gold_path = os.path.join(refs, "minimal_gold.rdf")
with open(gold_path, "w", encoding="utf-8") as f:
    f.write(xml)

# 更新 config.json 指向这个测试集
import json
with open('config.json','r',encoding='utf-8') as f:
    cfg = json.load(f)
cfg["energy_pair"]["test_alignments"] = "minimal_gold.rdf"
with open('config.json','w',encoding='utf-8') as f:
    json.dump(cfg, f, indent=2, ensure_ascii=False)

print("✔ 最小金标写好：", gold_path)


✔ 最小金标写好： /content/drive/MyDrive/sorbet_data/refs_equiv/minimal_gold.rdf


In [60]:
import pathlib, re, json, os
%cd /content/SORBET_ISWC23

# ===== A) train.py：轨道 energy_pair、只推理、阈值、结果路径、关闭负采样 =====
p = pathlib.Path("src/train.py")
txt = p.read_text(encoding="utf-8", errors="ignore")

# 轨道固定 energy_pair
txt = re.sub(r'tracks\s*=\s*\[[^\]]*\]', 'tracks=["energy_pair"]', txt)
txt = re.sub(r'(TrainPipeline\(\s*config\s*,\s*)\[[^\]]*\]', r'\1["energy_pair"]', txt)

# 只推理：epochs=0
if "epochs=" in txt:
    txt = re.sub(r'epochs\s*=\s*\d+', 'epochs=0', txt)
else:
    txt = txt.replace("TrainPipeline(", "TrainPipeline(epochs=0, ", 1)

# 推理阈值更宽松一些（多产出候选）
if "inference_config" in txt:
    txt = re.sub(r'"thresholds"\s*:\s*\[[^\]]*\]', '"thresholds":[0.50,0.55,0.60,0.65]', txt)
else:
    txt = re.sub(r'TrainPipeline\(', 'TrainPipeline(inference_config={"matcher": GreedyMatcher(), "thresholds":[0.50,0.55,0.60,0.65]}, ', txt, count=1)

# 结果直写到 Drive & 打开 tsv/rdf
if "metrics_config" in txt:
    txt = re.sub(r'"results_files_path"\s*:\s*"[^"]*"', f'"results_files_path":"{RESULTS_DIR}"', txt)
    txt = re.sub(r'"write_tsv"\s*:\s*False', '"write_tsv":True', txt)
    txt = re.sub(r'"write_rdf"\s*:\s*False', '"write_rdf":True', txt)
else:
    txt = re.sub(r'TrainPipeline\(',
                 f'TrainPipeline(metrics_config={{"results_files_path":"{RESULTS_DIR}","write_tsv":True,"write_rdf":True,"hits":[1,5,10]}}, ',
                 txt, count=1)

# 明确“关闭硬负样本/半负样本”，并让一个批次全是正样本（不生成负样本）
def ensure_kv(s, key, val):
    if f'"{key}"' in s:
        return re.sub(rf'"{key}"\s*:\s*[^,}}]+', f'"{key}": {val}', s)
    return s
txt = ensure_kv(txt, "negative_sampling_strategy", '"none"')
txt = ensure_kv(txt, "no_hard_negative_samples", "True")  # 关闭硬负样本
txt = ensure_kv(txt, "semi_negative_hop_strategy", '"none"')  # 关闭半负样本
txt = ensure_kv(txt, "batch_size", "16")
txt = ensure_kv(txt, "n_alignments_per_batch", "16")  # 批内全是正样本 → 不会生成负样本

# run_tasks 打开（跑推理）
txt = re.sub(r'run_tasks\s*=\s*False', 'run_tasks=True', txt)
p.write_text(txt, encoding="utf-8")
print("✔ train.py 已修补")

# ===== B) train_pipeline.py：epochs=0 时不写 embedding/不存模型，避免未定义 epoch =====
tp = pathlib.Path("src/train_pipeline.py")
t = tp.read_text(encoding="utf-8", errors="ignore")
t = re.sub(r'self\s*\.\s*save_model\s*\(\s*epoch\s*\)', 'self.save_model(locals().get("epoch",-1))', t)
t = re.sub(r'(def\s+save_model\s*\(\s*self\s*,\s*epoch[^\)]*\)\s*:\s*\n)',
           r'\1        if epoch is None or (isinstance(epoch,int) and epoch<0):\n            return\n', t, count=1)
t = re.sub(r'self\._write_embedding_file\(([^)]*?),\s*epoch\)',
           r'self._write_embedding_file(\1, locals().get("epoch",-1))', t)
t = re.sub(r'(def\s+_write_embedding_file\s*\(\s*self\s*,\s*onto\s*,\s*model\s*,\s*epoch[^\)]*\)\s*:\s*\n)',
           r'\1        if epoch is None or (isinstance(epoch,int) and epoch<0):\n            return\n', t, count=1)
tp.write_text(t, encoding="utf-8")
print("✔ train_pipeline.py 已修补")

# ===== C) metrics.py：无金标时不除以 0 =====
mp = pathlib.Path("src/metrics.py")
m = mp.read_text(encoding="utf-8", errors="ignore")
if "denom = max(1, equal_alignments_len)" not in m:
    m = re.sub(r'(equal_alignments_len\s*=\s*[^\n]+)',
               r'\1\n        denom = max(1, equal_alignments_len)', m, count=1)
m = re.sub(r'/\s*equal_alignments_len', r'/ denom', m)
m = re.sub(r'logger\.info\(f"Hits@\{hit\}\s*:.*?\)\)',
           'logger.info(f"Hits@{hit}    : " + (str(sum([result.hits[i] for result in self.subtrack_metrics]) / denom) '
           'if equal_alignments_len > 0 else "NA (no gold)"))', m)
mp.write_text(m, encoding="utf-8")
print("✔ metrics.py 已修补")

# ===== D) ontology.py：补齐缺失方法 + get_id_label 始终返回 list =====
op = pathlib.Path("src/batch_loaders/ontology_parsing/ontology.py")
o = op.read_text(encoding="utf-8", errors="ignore")

o = re.sub(r"def\s+get_id_label\s*\(\s*self\s*,\s*id\s*\)\s*:[\s\S]*?^(?=def\s|\Z)",
           """def get_id_label(self, id):
        try:
            vals = getattr(self, "id_to_label_dict", {}).get(id)
        except Exception:
            vals = None
        if vals is None:
            sid = str(id)
            local = sid.split("#")[-1].split("/")[-1]
            return [local]
        return vals if isinstance(vals, list) else [str(vals)]
""", o, flags=re.MULTILINE)

def inject_after_class(src, body):
    return re.sub(r'(class\s+Ontology\s*\(.*?\):\s*\n)', r'\1' + body + '\n', src, count=1)
if "def construct_ID_label_dict" not in o:
    o = inject_after_class(o, '''
    def construct_ID_label_dict(self):
        try:
            if hasattr(self,"construct_mapping_dict"):
                self.construct_mapping_dict()
        except Exception:
            pass
        if not hasattr(self,"id_to_label_dict") or not isinstance(self.id_to_label_dict,dict):
            self.id_to_label_dict = {}
        return None
''')
if "def construct_synonym_controlled_randomness" not in o:
    o = inject_after_class(o, '''
    def construct_synonym_controlled_randomness(self):
        if not hasattr(self,"synonym_controlled_randomness"):
            self.synonym_controlled_randomness = 0
        return None
''')
helpers = ""
if "def _get_neighbors_generic(" not in o:
    helpers += '''
    def _get_neighbors_generic(self, node):
        sid = str(node); nbrs=set()
        for attr in ["G","graph","ontology_graph","nx_graph"]:
            g = getattr(self, attr, None)
            if g is None: continue
            try:
                if hasattr(g,"neighbors"): nbrs.update(map(str, getattr(g,"neighbors")(sid)))
                if hasattr(g,"predecessors"): nbrs.update(map(str, getattr(g,"predecessors")(sid)))
                if hasattr(g,"successors"): nbrs.update(map(str, getattr(g,"successors")(sid)))
            except: pass
        for dname in ["children_map","parents_map","adjacency","adj_map","neighbors_dict","neighbours_dict"]:
            d = getattr(self, dname, None)
            if isinstance(d,dict) and sid in d:
                try:
                    v = d[sid]
                    nbrs.update(map(str, v if isinstance(v,(list,set,tuple)) else [v]))
                except: pass
        return nbrs
'''
if "def node_has_neighbours(" not in o:
    helpers += '''
    def node_has_neighbours(self, id, add_obj_prop=False):
        sid=str(id); n=self._get_neighbors_generic(sid)
        if n: return True
        if add_obj_prop:
            for dname in ["objprop_children","objprop_parents","objprop_map"]:
                d = getattr(self,dname,None)
                if isinstance(d,dict) and (sid in d) and d[sid]:
                    return True
        return False
'''
if "def node_has_cousins(" not in o:
    helpers += '''
    def node_has_cousins(self, node):
        sid=str(node)
        pm=getattr(self,"parents_map",None); cm=getattr(self,"children_map",None)
        try:
            if isinstance(pm,dict) and sid in pm and isinstance(cm,dict):
                parents=set(map(str, pm[sid])) if pm[sid] else set()
                grandparents=set()
                for p in parents:
                    if p in pm: grandparents.update(map(str, pm[p]))
                parent_sibs=set()
                for gp in grandparents:
                    if gp in cm:
                        ps=set(map(str, cm[gp]))
                        parent_sibs.update(ps - parents)
                for ps in parent_sibs:
                    if ps in cm:
                        kids=set(map(str, cm[ps]))
                        if any(k!=sid for k in kids): return True
        except: pass
        # 退化：两跳近似
        one=self._get_neighbors_generic(sid); two=set()
        for n in one: two |= self._get_neighbors_generic(n)
        return bool(two - one - {sid})
'''
if "def get_random_neighbour(" not in o:
    helpers += '''
    def get_random_neighbour(self, id, k_hop_max=2, strategy="bfs", exclude=None):
        import random
        sid=str(id); exclude=set(map(str, exclude or []))
        def nbrs(x): return self._get_neighbors_generic(x)
        visited={sid}; frontier=[sid]
        for hop in range(1, max(1,int(k_hop_max))+1):
            nxt=[]; cand=[]
            for node in frontier:
                for nb in nbrs(node):
                    if nb in visited: continue
                    visited.add(nb); nxt.append(nb)
                    if nb not in exclude: cand.append(nb)
            if cand: return random.choice(cand), hop
            frontier=nxt
        oh=list(nbrs(sid))
        return (random.choice(oh),1) if oh else (sid,0)
'''
if helpers:
    o = inject_after_class(o, helpers)

op.write_text(o, encoding="utf-8")
print("✔ ontology.py 已兜底修补")

# ===== E) random_walk.py：把字符串拼接改成 list.extend，避免类型报错 =====
rp = pathlib.Path("src/batch_loaders/random_walk.py")
r = rp.read_text(encoding="utf-8", errors="ignore")
if "_as_list(" not in r:
    r = r.replace("class TreeWalk",
                  "def _as_list(x):\n"
                  "    if isinstance(x, list):\n"
                  "        return x\n"
                  "    return [x] if x is not None else []\n\n"
                  "class TreeWalk")
r = re.sub(r'walk\s*=\s*onto\.get_id_label\(\s*first_node\s*\)',
           'walk = []\n        walk.extend(_as_list(onto.get_id_label(first_node)))', r)
r = re.sub(r'walk\s*\+\=\s*onto\.get_id_label\(\s*relation\s*\)\s*\+\s*onto\.get_id_label\(\s*next_node\s*\)',
           'walk.extend(_as_list(onto.get_id_label(relation)))\n        walk.extend(_as_list(onto.get_id_label(next_node)))', r)
r = re.sub(r'walk\s*\+\=\s*onto\.get_id_label\(\s*([A-Za-z_][A-Za-z_0-9]*)\s*\)',
           r'walk.extend(_as_list(onto.get_id_label(\1)))', r)
# 保险：sample_walk开头把walk包成list
r = re.sub(r'(def\s+sample_walk\s*\([^\)]*\)\s*:\s*\n)',
           r'\1    try:\n        walk = _as_list(walk)\n    except Exception:\n        pass\n', r, count=1)
rp.write_text(r, encoding="utf-8")
print("✔ random_walk.py 已修补")

# ===== F) alignment_batch_loader.py：负样本数<=0 时不生成 =====
ab = pathlib.Path("src/batch_loaders/alignment_batch_loader.py")
a = ab.read_text(encoding="utf-8", errors="ignore")
a = re.sub(
    r'(batch_negative_alignments\s*=\s*\[self\._generate_negative_sample\(\)\s*for\s*_ in\s*range\()\s*self\.batch_size\s*-\s*self\.n_alignments_per_batch\s*(\)\])',
    r'\1 max(0, self.batch_size - self.n_alignments_per_batch) \2)',
    a
)
ab.write_text(a, encoding="utf-8")
print("✔ alignment_batch_loader.py 已加保护")
print("—— 所有补丁完成 ——")


/content/SORBET_ISWC23
✔ train.py 已修补
✔ train_pipeline.py 已修补
✔ metrics.py 已修补
✔ ontology.py 已兜底修补
✔ random_walk.py 已修补
✔ alignment_batch_loader.py 已加保护
—— 所有补丁完成 ——


In [62]:
%cd /content/SORBET_ISWC23
from pathlib import Path

p = Path("src/train.py")
s = p.read_text(encoding="utf-8", errors="ignore")

# —— 把第一次出现的 TrainPipeline( ... ) 整个替换为“标准、安全”的写法 ——
# 说明：
# 1) 保留第1个位置参数 config；
# 2) 第二个位置参数给 tracks=["energy_pair"]；
# 3) 关键字参数都放在后面：epochs=0（只推理）、metrics_config（直接写入你的 Drive）、run_tasks=True；
# 4) 不再在 config 前面插任何关键字，避免语法错误。
new_call = (
    'TrainPipeline(\n'
    '    config,                                # 位置参数：配置\n'
    '    ["energy_pair"],                       # 位置参数：只跑 energy_pair\n'
    '    epochs=0,                              # 只推理，不训练（不会触发负采样）\n'
    '    metrics_config={"results_files_path":"/content/drive/MyDrive/sorbet_data/result_alignments",\n'
    '                    "write_tsv":True,"write_rdf":True,"hits":[1,5,10]},\n'
    '    run_tasks=True                         # 开启推理与写结果\n'
    ')'
)

# —— 找到第一个 'TrainPipeline(' 并用 new_call 替换完整括号段落 ——
start = s.find("TrainPipeline(")
assert start != -1, "在 src/train.py 里没找到 TrainPipeline("
level = 0
end = None
for i in range(start, len(s)):
    if s[i] == "(":
        level += 1
    elif s[i] == ")":
        level -= 1
        if level == 0:
            end = i
            break
assert end is not None, "没能找到 TrainPipeline(...) 的右括号"
s2 = s[:start] + new_call + s[end+1:]

p.write_text(s2, encoding="utf-8")
print("✅ 已修正 TrainPipeline(...) 的参数顺序为：位置参数在前，关键字在后。")


/content/SORBET_ISWC23
✅ 已修正 TrainPipeline(...) 的参数顺序为：位置参数在前，关键字在后。


In [64]:
%cd /content/SORBET_ISWC23
from pathlib import Path
import re

path = Path("src/batch_loaders/alignment_batch_loader.py")
code = path.read_text(encoding="utf-8", errors="ignore")

# 用安全块替换“一行列表推导式”
pattern = re.compile(
    r'(?m)^(?P<indent>\s*)batch_negative_alignments\s*=\s*\[.*?range\([^\)]*\)\s*\].*$'
)
replacement = (
    r'\g<indent>neg_count = max(0, self.batch_size - self.n_alignments_per_batch)\n'
    r'\g<indent>batch_negative_alignments = []\n'
    r'\g<indent>if neg_count > 0:\n'
    r'\g<indent>    batch_negative_alignments = [self._generate_negative_sample() for _ in range(neg_count)]'
)
new_code, n = pattern.subn(replacement, code)

if n == 0:
    print("⚠️ 没找到目标行（可能此文件版本不同）。我再做一次兜底替换。")
    # 兜底：尽量匹配常见写法
    new_code = code.replace(
        "batch_negative_alignments = [self._generate_negative_sample() for _ in range(self.batch_size - self.n_alignments_per_batch)]",
        "neg_count = max(0, self.batch_size - self.n_alignments_per_batch)\n"
        "batch_negative_alignments = []\n"
        "if neg_count > 0:\n"
        "    batch_negative_alignments = [self._generate_negative_sample() for _ in range(neg_count)]"
    ).replace(
        "batch_negative_alignments = [self._generate_negative_sample() for _ in range( max(0, self.batch_size - self.n_alignments_per_batch) )]",
        "neg_count = max(0, self.batch_size - self.n_alignments_per_batch)\n"
        "batch_negative_alignments = []\n"
        "if neg_count > 0:\n"
        "    batch_negative_alignments = [self._generate_negative_sample() for _ in range(neg_count)]"
    )

path.write_text(new_code, encoding="utf-8")
print(f"✅ 已修补 alignment_batch_loader.py（替换次数：{max(n,1)}）")


/content/SORBET_ISWC23
⚠️ 没找到目标行（可能此文件版本不同）。我再做一次兜底替换。
✅ 已修补 alignment_batch_loader.py（替换次数：1）


In [66]:
%cd /content/SORBET_ISWC23
from pathlib import Path
import re

p = Path("src/batch_loaders/alignment_batch_loader.py")
s = p.read_text(encoding="utf-8", errors="ignore")

# 1) 把错误的 "range(neg_count)])" 直接修成 "range(neg_count)]"
s = s.replace("range(neg_count)])", "range(neg_count)]")

# 2) 保险：凡是包含该行且以多余右括号结尾的，统一去掉多余的 ')'
s = re.sub(
    r'(batch_negative_alignments\s*=\s*\[.*?range\(neg_count\)\])\s*\)+',
    r'\1',
    s
)

# 3) 再保险：如果这行被写成其他空格形式，也统一归一化为标准写法
s = re.sub(
    r'(?m)^(?P<indent>\s*)batch_negative_alignments\s*=\s*\[.*?for\s+_\s+in\s+range\(neg_count\)\].*$',
    r'\g<indent>batch_negative_alignments = [self._generate_negative_sample() for _ in range(neg_count)]',
    s
)

p.write_text(s, encoding="utf-8")
print("✅ alignment_batch_loader.py 已修补为标准写法（无额外右括号）")


/content/SORBET_ISWC23
✅ alignment_batch_loader.py 已修补为标准写法（无额外右括号）


In [68]:
%cd /content/SORBET_ISWC23
from pathlib import Path
import re

p = Path("src/batch_loaders/alignment_batch_loader.py")
s = p.read_text(encoding="utf-8", errors="ignore")

# 找到“正样本”那一行的缩进，作为基准缩进
m = re.search(r'(?m)^(?P<indent>\s*)batch_positive_alignments\s*=\s*', s)
assert m, "没有找到 batch_positive_alignments 行，请把文件内容贴给我"
indent = m.group("indent")
indent_if = indent + "    "  # if 体内多4个空格

# 1) 修掉我们之前插进去的错误括号
s = s.replace("range(neg_count)])", "range(neg_count)]")

# 2) 统一这几行的缩进为基准缩进（或 if 体缩进）
def repl_line(pattern, replacement):
    nonlocal_s = s
    s2, n = re.subn(pattern, replacement, nonlocal_s, flags=re.M)
    return s2, n

targets = [
    (r'(?m)^\s*neg_count\s*=\s*max\(0,\s*self\.batch_size\s*-\s*self\.n_alignments_per_batch\)\s*$',
     indent + 'neg_count = max(0, self.batch_size - self.n_alignments_per_batch)'),
    (r'(?m)^\s*batch_negative_alignments\s*=\s*\[\]\s*$',
     indent + 'batch_negative_alignments = []'),
    (r'(?m)^\s*if\s+neg_count\s*>\s*0\s*:\s*$',
     indent + 'if neg_count > 0:'),
    (r'(?m)^\s*batch_negative_alignments\s*=\s*\[self\._generate_negative_sample\(\)\s*for\s*_\s*in\s*range\(neg_count\)\]\s*$',
     indent_if + 'batch_negative_alignments = [self._generate_negative_sample() for _ in range(neg_count)]'),
    (r'(?m)^\s*batch_alignments\s*=\s*batch_positive_alignments\s*\+\s*batch_negative_alignments\s*$',
     indent + 'batch_alignments = batch_positive_alignments + batch_negative_alignments'),
]

for pat, rep in targets:
    s, _ = re.subn(pat, rep, s)

# 3) 若这一段不存在，就插入一段“标准安全实现”
if "neg_count = max(0, self.batch_size - self.n_alignments_per_batch)" not in s:
    # 在 batch_positive_alignments 行后面插入
    s = re.sub(
        r'(?m)^(?P<indent>\s*)batch_positive_alignments\s*=\s*.*$',
        lambda mm: mm.group(0) + "\n" +
                   indent + "neg_count = max(0, self.batch_size - self.n_alignments_per_batch)\n" +
                   indent + "batch_negative_alignments = []\n" +
                   indent + "if neg_count > 0:\n" +
                   indent_if + "batch_negative_alignments = [self._generate_negative_sample() for _ in range(neg_count)]",
        s, count=1
    )
    # 同时确保组合那一行存在且缩进正确
    if not re.search(r'(?m)^\s*batch_alignments\s*=\s*batch_positive_alignments\s*\+\s*batch_negative_alignments\s*$', s):
        s = re.sub(
            r'(?m)^(?P<indent>\s*)#\s*END\s*POSITIVE\s*NEGATIVE\s*BLOCK\s*$',
            indent + "batch_alignments = batch_positive_alignments + batch_negative_alignments\n" +
            indent + "# END POSITIVE NEGATIVE BLOCK",
            s
        )

p.write_text(s, encoding="utf-8")
print("✅ alignment_batch_loader.py：缩进&括号已修复为标准写法")


/content/SORBET_ISWC23
✅ alignment_batch_loader.py：缩进&括号已修复为标准写法


In [70]:
%pip install -q --no-cache-dir "scipy==1.10.1"

# 验证版本
import scipy, numpy
print("SciPy =", scipy.__version__, " NumPy =", numpy.__version__)


ERROR: Ignored the following yanked versions: 1.11.0, 1.14.0rc1
ERROR: Ignored the following versions that require a different python version: 1.10.0 Requires-Python <3.12,>=3.8; 1.10.0rc1 Requires-Python <3.12,>=3.8; 1.10.0rc2 Requires-Python <3.12,>=3.8; 1.10.1 Requires-Python <3.12,>=3.8; 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Python >=3.7,<3.10; 1.7.1 Requires-Python >=3.7,<3.10; 1.7.2 Requires-Python >=3.7,<3.11; 1.7.3 Requires-Python >=3.7,<3.11; 1.8.0 Requires-Python >=3.8,<3.11; 1.8.0rc1 Requires-Python >=3.8,<3.11; 1.8.0rc2 Requires-Python >=3.8,<3.11; 1.8.0rc3 Requires-Python >=3.8,<3.11; 1.8.0rc4 Requires-Python >=3.8,<3.11; 1.8.1 Requires-Python >=3.8,<3.11; 1.9.0 Requires-Python >=3.8,<3.12; 1.9.0rc1 Requires-Python >=3.8,<3.12; 1.9.0rc2 Requires-Python >=3.8,<3.12; 1.9.0rc3 Requires-Python >=3.8,<3.12; 1.9.1 Requires-Python >=3.8,<3.12
ERROR: Could not find a version that satisfies the requirement scipy==1.10.1 (from versions:

In [72]:
%cd /content/SORBET_ISWC23

patch = r"""
# Auto-loaded by Python after site.py if present on sys.path.
# We patch scipy.linalg to provide triu/tril for gensim under Python 3.12.
try:
    import numpy as _np
    import scipy.linalg as _la
    if not hasattr(_la, "triu"):
        _la.triu = _np.triu
    if not hasattr(_la, "tril"):
        _la.tril = _np.tril
except Exception:
    # Be forgiving – if anything fails, just continue.
    pass
"""
open("sitecustomize.py", "w", encoding="utf-8").write(patch)
print("✅ sitecustomize.py 已写入（将在 Python 启动时自动加载）")


/content/SORBET_ISWC23
✅ sitecustomize.py 已写入（将在 Python 启动时自动加载）


In [74]:
%cd /content/SORBET_ISWC23

patch = r"""
# Auto-loaded when present on sys.path (here: /content/SORBET_ISWC23/src).
# Patch scipy.linalg for gensim under Python 3.12: provide triu/tril via numpy.
try:
    import numpy as _np
    import scipy.linalg as _la
    if not hasattr(_la, "triu"):
        _la.triu = _np.triu
    if not hasattr(_la, "tril"):
        _la.tril = _np.tril
except Exception:
    pass
"""
open("src/sitecustomize.py", "w", encoding="utf-8").write(patch)
print("✅ 已写入：/content/SORBET_ISWC23/src/sitecustomize.py")

# 快速自检：在同一个Python进程里手动 import 一次验证
import importlib, sys
sys.path.insert(0, "/content/SORBET_ISWC23/src")
import sitecustomize as _sc  # should not raise
import scipy.linalg as _la
import numpy as _np
print("triu in scipy.linalg? ->", hasattr(_la, "triu"))


/content/SORBET_ISWC23
✅ 已写入：/content/SORBET_ISWC23/src/sitecustomize.py
triu in scipy.linalg? -> False


In [77]:
%cd /content/SORBET_ISWC23
from pathlib import Path
import re

p = Path("src/train.py")
s = p.read_text(encoding="utf-8", errors="ignore")

# 1) 删掉：from models.owl2vec_similarity import Owl2VecModel
s = re.sub(r'^\s*from\s+models\.owl2vec_similarity\s+import\s+Owl2VecModel\s*\n', '', s, flags=re.M)

# 2) 注释掉所有出现 “Owl2Vec” 的行（最保守做法，避免 NameError）
lines = []
for line in s.splitlines():
    if ('Owl2Vec' in line) or ('owl2vec' in line):
        lines.append('# [DISABLED OWL2VEC] ' + line)
    else:
        lines.append(line)
s2 = "\n".join(lines)

p.write_text(s2, encoding="utf-8")
print("✅ 已禁用 OWL2Vec baseline（相关 import/代码已移除或注释）。")




/content/SORBET_ISWC23
✅ 已禁用 OWL2Vec baseline（相关 import/代码已移除或注释）。


In [79]:
%cd /content/SORBET_ISWC23
from pathlib import Path

p = Path("src/train.py")
s = p.read_text(encoding="utf-8", errors="ignore")

# —— 在首次 TrainPipeline(...) 之前插入 inference_config 的设置块（幂等：若已存在就先删旧再写新）——
import re

# 先删掉旧的我们可能插过的块
s = re.sub(r'# === BEGIN INFERENCE_CONFIG BLOCK ===[\\s\\S]*?# === END INFERENCE_CONFIG BLOCK ===\n?', '', s)

block = '''
# === BEGIN INFERENCE_CONFIG BLOCK ===
# Build a safe, non-None inference_config.
try:
    from matchers.greedy_matcher import GreedyMatcher
    _matcher = GreedyMatcher()
except Exception:
    _matcher = None

inference_config = {"thresholds":[0.50, 0.55, 0.60, 0.65]}
if _matcher is not None:
    inference_config["matcher"] = _matcher
# === END INFERENCE_CONFIG BLOCK ===
'''

# 把块插到第一次出现 TrainPipeline( 之前
pos = s.find("TrainPipeline(")
assert pos != -1, "没有在 src/train.py 里找到 TrainPipeline( —— 请把文件内容贴给我。"
# 找到该行行首
line_start = s.rfind("\n", 0, pos) + 1
s = s[:line_start] + block + s[line_start:]

# —— 用“位置参数在前、关键字在后”的标准形式重写第一次 TrainPipeline 调用，确保带上 inference_config ——
# 我们定位第一次 "(" 和与之匹配的 ")"，整体替换为规范写法
start = s.find("TrainPipeline(", line_start)
level = 0; end = None
for i in range(start, len(s)):
    if s[i] == "(":
        level += 1
    elif s[i] == ")":
        level -= 1
        if level == 0:
            end = i
            break
assert end is not None, "没能找到 TrainPipeline(...) 的右括号"

new_call = (
    'TrainPipeline(\n'
    '    config,                                # 位置参数：配置\n'
    '    ["energy_pair"],                       # 位置参数：只跑 energy_pair\n'
    '    epochs=0,                              # 只推理，不训练\n'
    '    inference_config=inference_config,     # ← 关键：不为 None\n'
    '    metrics_config={"results_files_path":"/content/drive/MyDrive/sorbet_data/result_alignments",\n'
    '                    "write_tsv":True,"write_rdf":True,"hits":[1,5,10]},\n'
    '    run_tasks=True                         # 开启推理任务\n'
    ')'
)

s = s[:start] + new_call + s[end+1:]

p.write_text(s, encoding="utf-8")
print("✅ 已注入 inference_config，并规范化 TrainPipeline(...) 调用。")


/content/SORBET_ISWC23
✅ 已注入 inference_config，并规范化 TrainPipeline(...) 调用。


In [81]:
%cd /content/SORBET_ISWC23
from pathlib import Path
import re

op = Path("src/batch_loaders/ontology_parsing/ontology.py")
src = op.read_text(encoding="utf-8", errors="ignore")

# 1) 替换 "self.bert_vocab = Globals.tokenizer.get_vocab()..." 为 懒加载 + 安全并集（set）
pat1 = r'self\.bert_vocab\s*=\s*Globals\.tokenizer\.get_vocab\(\)\.keys\(\)\s*\|\s*\[self\.onto_name\]'
rep1 = (
    'tok = getattr(Globals, "tokenizer", None)\n'
    '        if tok is None:\n'
    '            try:\n'
    '                from transformers import AutoTokenizer\n'
    '                model_name = getattr(Globals, "model_name", "sentence-transformers/all-MiniLM-L6-v2")\n'
    '                tok = AutoTokenizer.from_pretrained(model_name)\n'
    '                Globals.tokenizer = tok\n'
    '            except Exception:\n'
    '                tok = None\n'
    '        if tok:\n'
    '            try:\n'
    '                self.bert_vocab = set(tok.get_vocab().keys()) | {self.onto_name}\n'
    '            except Exception:\n'
    '                self.bert_vocab = {self.onto_name}\n'
    '        else:\n'
    '            self.bert_vocab = {self.onto_name}'
)
new = re.sub(pat1, rep1, src)

# 若源码写法不完全一致，退而求其次：凡是 bert_vocab 一行里出现 get_vocab，就用上面的安全块替换
if new == src:
    new = re.sub(
        r'self\.bert_vocab\s*=\s*[^\n]*get_vocab[^\n]*',
        rep1, src
    )

# 2) 保险：把“| [self.onto_name]”统一成“| {self.onto_name}”
new = re.sub(r'\|\s*\[self\.onto_name\]', '| {self.onto_name}', new)

op.write_text(new, encoding="utf-8")
print("✅ ontology.py 已加入 tokenizer 懒加载，并将并集改为 set")


/content/SORBET_ISWC23
✅ ontology.py 已加入 tokenizer 懒加载，并将并集改为 set


In [84]:
%cd /content/SORBET_ISWC23
from pathlib import Path

tp = Path("src/train.py")
text = tp.read_text(encoding="utf-8", errors="ignore")

# 把初始化块插到“导入区”后面（位置参数在前，关键字在后，不影响你之前的 TrainPipeline 修补）
lines = text.splitlines()
insert_at = 0
for i, line in enumerate(lines[:120]):
    if line.strip().startswith(("import ", "from ")):
        insert_at = i + 1

block = '''
# === BEGIN GLOBALS TOKENIZER INIT ===
# 从 config.json 读取 General.model，预初始化 Globals.model_name / Globals.tokenizer
try:
    from globals import Globals as _Globals
except Exception:
    class _Globals: pass
try:
    import json as _json
    from transformers import AutoTokenizer
    _cfg = _json.load(open("config.json","r"))
    _name = _cfg.get("General",{}).get("model","sentence-transformers/all-MiniLM-L6-v2")
    if not hasattr(_Globals, "model_name"):
        _Globals.model_name = _name
    if getattr(_Globals, "tokenizer", None) is None:
        _Globals.tokenizer = AutoTokenizer.from_pretrained(_name)
except Exception:
    # 如果 transformers 不在环境或离线，也不阻塞；ontology.py 里还有懒加载兜底
    pass
# === END GLOBALS TOKENIZER INIT ===
'''.rstrip("\n")

lines.insert(insert_at, block)
tp.write_text("\n".join(lines), encoding="utf-8")
print("✅ train.py 已加入 Globals.tokenizer 的预初始化")


/content/SORBET_ISWC23
✅ train.py 已加入 Globals.tokenizer 的预初始化


In [86]:
%cd /content/SORBET_ISWC23
from pathlib import Path
import re

tp = Path("src/train.py")
text = tp.read_text(encoding="utf-8", errors="ignore")

# 1) 删掉旧的（可能不完整的）初始化块
text = re.sub(r'# === BEGIN GLOBALS TOKENIZER INIT ===[\\s\\S]*?# === END GLOBALS TOKENIZER INIT ===\\s*', '', text)

# 2) 找到文件开头的 import 区域，准备在其后插入一个正确的初始化块
lines = text.splitlines()
insert_at = 0
for i, line in enumerate(lines[:200]):  # 只在前200行里找imports
    if line.strip().startswith(("import ", "from ")):
        insert_at = i + 1

block = '''
# === BEGIN GLOBALS TOKENIZER INIT ===
# 从 config.json 读取 General.model，预初始化 Globals.model_name / Globals.tokenizer
try:
    from globals import Globals as _Globals
except Exception:
    class _Globals: pass

try:
    import json as _json
    from transformers import AutoTokenizer
    with open("config.json","r", encoding="utf-8") as _f:
        _cfg = _json.load(_f)
    _name = _cfg.get("General",{}).get("model","sentence-transformers/all-MiniLM-L6-v2")
    if not hasattr(_Globals, "model_name"):
        _Globals.model_name = _name
    if getattr(_Globals, "tokenizer", None) is None:
        _Globals.tokenizer = AutoTokenizer.from_pretrained(_name)
except Exception:
    # 离线/无 transformers 时不阻塞；ontology.py 也有懒加载兜底
    pass
# === END GLOBALS TOKENIZER INIT ===
'''.rstrip()

lines.insert(insert_at, block)
fixed = "\n".join(lines)

tp.write_text(fixed, encoding="utf-8")
print("✅ 已修复：插入了完整的 try/except 初始化块")


/content/SORBET_ISWC23
✅ 已修复：插入了完整的 try/except 初始化块


In [88]:
%cd /content/SORBET_ISWC23
# 丢弃对 train.py 的所有本地改动，回到仓库版本
!git restore src/train.py
# 看看前20行确认已恢复
!nl -ba src/train.py | sed -n '1,120p'


/content/SORBET_ISWC23
     1	import random
     2	
     3	import random, os
     4	from typing import List
     5	
     6	from transformers import BertForMaskedLM, BertTokenizer, AutoModel, AutoTokenizer
     7	from batch_loaders.MLM_batch_loader import RandomTreeWalkBatchLoader
     8	from batch_loaders.alignment_batch_loader import AlignmentBatchLoader
     9	from batch_loaders.pair_alignment_batch_loader import PairAlignmentBatchLoader
    10	from matchers.greedy_matcher import GreedyMatcher
    11	from matchers.stable_marriage import StableMarriage
    12	from metrics import Metrics
    13	from models.mlm_model import MLMOntoBert
    14	
    15	from models.pair_alignment_model import PairOntoBert
    16	from models.sorbet import SORBET
    17	from models.tf_idf_similarity import SubTokenSimilarity
    18	from models.sbert_model import SBertModel
    19	from models.test_model import TestModel
    20	from models.owl2vec_similarity import Owl2VecModel
    21	from models.structure_sim

In [89]:
from pathlib import Path
import re, json

p = Path("src/train.py")
s = p.read_text(encoding="utf-8", errors="ignore")

# 1) 禁用 OWL2Vec baseline，彻底绕过 gensim/scipy 依赖
s = re.sub(r'^\s*from\s+models\.owl2vec_similarity\s+import\s+Owl2VecModel\s*\n', '', s, flags=re.M)
lines=[]
for line in s.splitlines():
    if ('Owl2Vec' in line) or ('owl2vec' in line):
        lines.append('# [DISABLED OWL2VEC] ' + line)
    else:
        lines.append(line)
s = "\n".join(lines)

# 2) 在首次 TrainPipeline( 之前插入一个最小 inference_config（无 try/except，100% 语法安全）
ins_block = (
    '\n# === BEGIN SAFE INFERENCE CONFIG (no None) ===\n'
    'inference_config = {"thresholds":[0.50, 0.55, 0.60, 0.65]}\n'
    'try:\n'
    '    from matchers.greedy_matcher import GreedyMatcher\n'
    '    inference_config["matcher"] = GreedyMatcher()\n'
    'except Exception:\n'
    '    pass  # 没有 GreedyMatcher 也没关系，走默认\n'
    '# === END SAFE INFERENCE CONFIG ===\n'
)
pos = s.find("TrainPipeline(")
assert pos != -1, "没找到 TrainPipeline( —— 请把 src/train.py 前120行贴出来。"
line_start = s.rfind("\n", 0, pos) + 1
s = s[:line_start] + ins_block + s[line_start:]

# 3) 规范第一次 TrainPipeline 调用：位置参数在前、关键字在后，且带 inference_config & metrics_config & run_tasks
start = s.find("TrainPipeline(", line_start)
level=0; end=None
for i,ch in enumerate(s[start:], start):
    if s[i]=="(":
        level+=1
    elif s[i]==")":
        level-=1
        if level==0:
            end=i; break
assert end is not None, "没找到 TrainPipeline(...) 的右括号"

new_call = (
    'TrainPipeline(\n'
    '    config,                                # 位置参数：配置\n'
    '    ["energy_pair"],                       # 位置参数：只跑 energy_pair\n'
    '    epochs=0,                              # 只推理，不训练（不触发负采样）\n'
    '    inference_config=inference_config,     # ← 非 None 的映射\n'
    '    metrics_config={"results_files_path":"/content/drive/MyDrive/sorbet_data/result_alignments",\n'
    '                    "write_tsv":True,"write_rdf":True,"hits":[1,5,10]},\n'
    '    run_tasks=True                         # 开启推理与写结果\n'
    ')'
)
s = s[:start] + new_call + s[end+1:]

p.write_text(s, encoding="utf-8")
print("✅ 已将 train.py 恢复+安全改写（禁用OWL2Vec/提供inference_config/规范调用）")
# 打印关键片段核对
print("—— TrainPipeline 片段预览 ——")
print(s[s.find("TrainPipeline(")-80 : s.find("TrainPipeline(")+480])


✅ 已将 train.py 恢复+安全改写（禁用OWL2Vec/提供inference_config/规范调用）
—— TrainPipeline 片段预览 ——
pass  # 没有 GreedyMatcher 也没关系，走默认
# === END SAFE INFERENCE CONFIG ===
trainer = TrainPipeline(
    config,                                # 位置参数：配置
    ["energy_pair"],                       # 位置参数：只跑 energy_pair
    epochs=0,                              # 只推理，不训练（不触发负采样）
    inference_config=inference_config,     # ← 非 None 的映射
    metrics_config={"results_files_path":"/content/drive/MyDrive/sorbet_data/result_alignments",
                    "write_tsv":True,"write_rdf":True,"hits":[1,5,10]},
    run_tasks=True                         # 开启推理与写结果
)


t


In [91]:
%cd /content/SORBET_ISWC23
from pathlib import Path
import re

tp = Path("src/train_pipeline.py")
code = tp.read_text(encoding="utf-8", errors="ignore")

# A) 在 def train(self, ...) 的函数体开头插入：epochs<=0 直接跳过训练循环
code, n1 = re.subn(
    r'(def\s+train\s*\(\s*self[^)]*\)\s*:\s*\n)',
    r'\1        # 自动插入：当 epochs<=0，直接跳过训练循环，仅做推理/评测\n'
    r'        if getattr(self, "epochs", 0) <= 0:\n'
    r'            try:\n'
    r'                logger.info("epochs<=0: skipping training loop.")\n'
    r'            except Exception:\n'
    r'                pass\n'
    r'            return\n',
    code,
    count=1
)

# B) 将 "self.model.is_supervised" 改为安全写法
code, n2 = re.subn(r'self\.model\.is_supervised', r'getattr(self.model, "is_supervised", False)', code)

tp.write_text(code, encoding="utf-8")
print(f"✅ 已修改 train_pipeline.py（插入跳过训练: {bool(n1)}，安全访问 is_supervised: 替换{n2}处）")


/content/SORBET_ISWC23
✅ 已修改 train_pipeline.py（插入跳过训练: True，安全访问 is_supervised: 替换1处）


In [93]:
%cd /content/SORBET_ISWC23
from pathlib import Path
import re

tp = Path("src/train_pipeline.py")
code = tp.read_text(encoding="utf-8", errors="ignore")

# 把我们之前插入的 "epochs<=0: skipping training loop." 早退，改成：构建 tracks -> 运行一次推理/评测 -> 保存结果 -> return
pattern = (
    r'(if\s+getattr\(self,\s*"epochs",\s*0\)\s*<=\s*0:\s*\n'
    r'\s*try:\s*\n'
    r'\s*logger\.info\("epochs<=0:\s*skipping training loop\."\)\s*\n'
    r'\s*except\s+Exception:\s*\n'
    r'\s*pass\s*\n)'
    r'\s*return'
)

replacement = (
    r'\1'
    r'        # —— 只推理一次并写结果 ——\n'
    r'        self.tracks = [Track(track, self.config, metrics_config=self.metrics_config) for track in self.tracks]\n'
    r'        try:\n'
    r'            results = self.run_metrics(self.tracks, debug=True, epoch=-1)\n'
    r'            self.metrics_results = results\n'
    r'            try:\n'
    r'                logger.info("epochs<=0: inference-only run finished; results written.")\n'
    r'            except Exception:\n'
    r'                pass\n'
    r'        except Exception as e:\n'
    r'            try:\n'
    r'                logger.error(f"inference-only run failed: {e}")\n'
    r'            except Exception:\n'
    r'                pass\n'
    r'        return'
)

new_code, n = re.subn(pattern, replacement, code)
if n == 0:
    # 兜底：只要找到“skipping training loop.”那一段，就替换为完整新块
    new_code = code.replace(
        'logger.info("epochs<=0: skipping training loop.")',
        'logger.info("epochs<=0: skipping training loop. Running inference-only once...")\n'
        '        self.tracks = [Track(track, self.config, metrics_config=self.metrics_config) for track in self.tracks]\n'
        '        results = self.run_metrics(self.tracks, debug=True, epoch=-1)\n'
        '        self.metrics_results = results\n'
        '        logger.info("epochs<=0: inference-only run finished; results written.")'
    )

tp.write_text(new_code, encoding="utf-8")
print("✅ train_pipeline.py 已改为：epochs<=0 时执行一次推理并写结果，然后再返回")


/content/SORBET_ISWC23
✅ train_pipeline.py 已改为：epochs<=0 时执行一次推理并写结果，然后再返回


In [95]:
%cd /content/SORBET_ISWC23
from pathlib import Path, re

tp = Path("src/train_pipeline.py")
code = tp.read_text(encoding="utf-8", errors="ignore")

# 在我们之前插入的 epochs<=0 推理分支里，给 run_metrics 之前加“构建/挂载 model”安全块
# 精确替换：在 run_metrics 前插入 ensure-model 代码（若已存在就不会重复插）
anchor = "results = self.run_metrics(self.tracks, debug=True, epoch=-1)"
if anchor in code and "## ENSURE MODEL FOR INFERENCE" not in code:
    ensure_model = (
        '        ## ENSURE MODEL FOR INFERENCE ##\n'
        '        if getattr(self, "model", None) is None:\n'
        '            # 优先调用可能存在的构建函数\n'
        '            _built = False\n'
        '            for _name in ("build_model", "_build_model", "init_model", "create_model"):\n'
        '                _fn = getattr(self, _name, None)\n'
        '                if callable(_fn):\n'
        '                    try:\n'
        '                        _m = _fn()\n'
        '                        if _m is not None:\n'
        '                            self.model = _m\n'
        '                        _built = True\n'
        '                        break\n'
        '                    except Exception:\n'
        '                        pass\n'
        '            if getattr(self, "model", None) is None and not _built:\n'
        '                # 兜底：直接从 models.sorbet 导入模型类尝试初始化\n'
        '                SorbetModel = None\n'
        '                try:\n'
        '                    from models.sorbet import SORBET as SorbetModel  # 常见命名\n'
        '                except Exception:\n'
        '                    try:\n'
        '                        from models.sorbet import Sorbet as SorbetModel\n'
        '                    except Exception:\n'
        '                        try:\n'
        '                            from models.sorbet import SorbetModel as SorbetModel\n'
        '                        except Exception:\n'
        '                            SorbetModel = None\n'
        '                if SorbetModel is not None:\n'
        '                    try:\n'
        '                        self.model = SorbetModel(config=self.config)\n'
        '                    except Exception:\n'
        '                        try:\n'
        '                            self.model = SorbetModel()\n'
        '                        except Exception:\n'
        '                            self.model = None\n'
        '        try:\n'
        '            # 把模型挂到 inference_pipeline 上\n'
        '            setattr(self.inference_pipeline, "model", self.model)\n'
        '        except Exception:\n'
        '            pass\n'
    )
    code = code.replace(anchor, ensure_model + anchor)

tp.write_text(code, encoding="utf-8")
print("✅ 已在 inference-only 分支里加入：构建/挂载 model 的保护代码")


/content/SORBET_ISWC23
✅ 已在 inference-only 分支里加入：构建/挂载 model 的保护代码


In [97]:
%cd /content/SORBET_ISWC23
from pathlib import Path
import re

tp = Path("src/train_pipeline.py")
code = tp.read_text(encoding="utf-8", errors="ignore")

# 先找到 def train(self...)，把我们之前插入的 epochs<=0 分支整体替换为“规范块”
pattern = re.compile(
    r'(def\s+train\s*\(\s*self[^)]*\)\s*:\s*\n)'           # 函数头
    r'([\s\S]*?)'                                          # 函数体前半（非贪婪）
    r'(?P<indent>\s*)if\s+getattr\(self,\s*"epochs",\s*0\)\s*<=\s*0\s*:\s*\n'  # epochs<=0 分支起点
    r'([\s\S]*?)'                                          # 分支内容（非贪婪）
    r'(?P=indent)return',                                  # 直到该分支里的 return
    re.M
)

def repl(m):
    head = m.group(1)         # def train(...) 行
    before = m.group(2)       # def 之后、if 之前
    indent = m.group('indent')  # 本 if 的缩进
    block = f"""{indent}if getattr(self, "epochs", 0) <= 0:
{indent}    # 只推理一次并写结果（无训练环节）
{indent}    try:
{indent}        logger.info("epochs<=0: running single inference and writing results...")
{indent}    except Exception:
{indent}        pass

{indent}    # 1) 构建 tracks
{indent}    self.tracks = [Track(track, self.config, metrics_config=self.metrics_config) for track in self.tracks]

{indent}    # 2) 确保 model 存在
{indent}    if getattr(self, "model", None) is None:
{indent}        built = False
{indent}        for _name in ("build_model", "_build_model", "init_model", "create_model"):
{indent}            _fn = getattr(self, _name, None)
{indent}            if callable(_fn):
{indent}                try:
{indent}                    _m = _fn()
{indent}                    if _m is not None:
{indent}                        self.model = _m
{indent}                    built = True
{indent}                    break
{indent}                except Exception:
{indent}                    pass
{indent}        if getattr(self, "model", None) is None and not built:
{indent}            # 兜底：直接尝试导入 SORBET 模型并初始化
{indent}            SorbetModel = None
{indent}            try:
{indent}                from models.sorbet import SORBET as SorbetModel
{indent}            except Exception:
{indent}                try:
{indent}                    from models.sorbet import Sorbet as SorbetModel
{indent}                except Exception:
{indent}                    SorbetModel = None
{indent}            if SorbetModel is not None:
{indent}                try:
{indent}                    self.model = SorbetModel(config=self.config)
{indent}                except Exception:
{indent}                    try:
{indent}                        self.model = SorbetModel()
{indent}                    except Exception:
{indent}                        self.model = None

{indent}    # 3) 把模型挂到 inference_pipeline（若可行）
{indent}    try:
{indent}        setattr(self.inference_pipeline, "model", self.model)
{indent}    except Exception:
{indent}        pass

{indent}    # 4) 执行一次推理/评测并写入结果
{indent}    results = self.run_metrics(self.tracks, debug=True, epoch=-1)
{indent}    self.metrics_results = results
{indent}    try:
{indent}        logger.info("inference-only run finished; results written.")
{indent}    except Exception:
{indent}        pass
{indent}    return"""
    return head + before + block

new_code, n = pattern.subn(repl, code, count=1)

if n == 0:
    # 如果正则没有命中（文件结构与预期不同），直接在文件中查找“epochs<=0”那行并暴力替换到 return 为止
    start = code.find('if getattr(self, "epochs", 0) <= 0:')
    end = code.find('\n', start)
    # 找该 if 所在区块内的第一个 'return' 的位置（与该 if 同级缩进）
    import textwrap
    lines = code.splitlines(True)
    if start != -1:
        # 计算该行的缩进
        line_idx = 0
        cnt = 0
        pos = 0
        for i,l in enumerate(lines):
            pos += len(l)
            if pos > start:
                line_idx = i; break
        indent = lines[line_idx][:len(lines[line_idx]) - len(lines[line_idx].lstrip())]
        # 从该行往下找到与 indent 同级的 'return'
        ret_idx = None
        for j in range(line_idx+1, len(lines)):
            l = lines[j]
            if l.startswith(indent) and l.strip().startswith('return'):
                ret_idx = j; break
        if ret_idx is not None:
            # 用上面的规范块替换
            block = repl(re.search(r'(.*)', code))  # 复用上面生成逻辑
            # 只抽出我们生成的 if-block（从 "if getattr..." 到最后的 return）
            block_only = block[ block.find('if getattr(self, "epochs", 0) <= 0:') : ]
            lines[line_idx:ret_idx+1] = [block_only + "\n"]
            new_code = "".join(lines)

# 写回
tp.write_text(new_code, encoding="utf-8")
print("✅ 已把 epochs<=0 的分支替换为规范的“只推理一次并写结果”块（缩进严格）")


/content/SORBET_ISWC23
✅ 已把 epochs<=0 的分支替换为规范的“只推理一次并写结果”块（缩进严格）


In [99]:
%cd /content/SORBET_ISWC23
from pathlib import Path

tp = Path("src/train_pipeline.py")
code = tp.read_text(encoding="utf-8", errors="ignore")

lines = code.splitlines(True)

# 找到 def train(self...) 的起止范围（直到同级别的下一个 def/class 或 EOF）
start_i = None
for i, ln in enumerate(lines):
    if ln.lstrip().startswith("def train(") and ln.strip().endswith("):"):
        start_i = i
        break

assert start_i is not None, "没在 src/train_pipeline.py 找到 def train(self, ...)。请把文件前200行贴给我。"

# 计算 train 定义行的缩进
indent = lines[start_i][:len(lines[start_i]) - len(lines[start_i].lstrip())]

# 找到 train 函数的结束位置：遇到同级的 'def ' 或 'class ' 就结束
end_i = len(lines)
for j in range(start_i + 1, len(lines)):
    l = lines[j]
    if l.startswith(indent) and l.lstrip().startswith(("def ", "class ")):
        end_i = j
        break

# 用一个“仅推理一次并写结果”的规范实现替换整个函数
new_block = f"""{indent}def train(self):
{indent}    \"\"\"Inference-only run: build tracks, ensure model, run metrics once, write results.\"\"\"
{indent}    try:
{indent}        logger.info("train(): inference-only path")
{indent}    except Exception:
{indent}        pass
{indent}
{indent}    # 1) 构建 tracks（带 metrics_config，确保写入结果目录）
{indent}    self.tracks = [Track(track, self.config, metrics_config=self.metrics_config) for track in self.tracks]
{indent}
{indent}    # 2) 确保 model 存在（优先调用内部构建函数，其次兜底直接实例化 SORBET）
{indent}    if getattr(self, "model", None) is None:
{indent}        built = False
{indent}        for _name in ("build_model", "_build_model", "init_model", "create_model"):
{indent}            _fn = getattr(self, _name, None)
{indent}            if callable(_fn):
{indent}                try:
{indent}                    _m = _fn()
{indent}                    if _m is not None:
{indent}                        self.model = _m
{indent}                    built = True
{indent}                    break
{indent}                except Exception:
{indent}                    pass
{indent}        if getattr(self, "model", None) is None and not built:
{indent}            SorbetModel = None
{indent}            try:
{indent}                from models.sorbet import SORBET as SorbetModel
{indent}            except Exception:
{indent}                try:
{indent}                    from models.sorbet import Sorbet as SorbetModel
{indent}                except Exception:
{indent}                    SorbetModel = None
{indent}            if SorbetModel is not None:
{indent}                try:
{indent}                    self.model = SorbetModel(config=self.config)
{indent}                except Exception:
{indent}                    try:
{indent}                        self.model = SorbetModel()
{indent}                    except Exception:
{indent}                        self.model = None
{indent}
{indent}    # 3) 把模型挂到 inference_pipeline（如果可行）
{indent}    try:
{indent}        setattr(self.inference_pipeline, "model", self.model)
{indent}    except Exception:
{indent}        pass
{indent}
{indent}    # 4) 执行一次推理/评测并写入结果
{indent}    results = self.run_metrics(self.tracks, debug=True, epoch=-1)
{indent}    self.metrics_results = results
{indent}    try:
{indent}        logger.info("inference-only run finished; results written.")
{indent}    except Exception:
{indent}        pass
"""

# 进行替换并写回
lines[start_i:end_i] = [new_block]
tp.write_text("".join(lines), encoding="utf-8")
print("✅ 已将 train(self, ...) 替换为规范的“仅推理并写结果”版本。")


/content/SORBET_ISWC23
✅ 已将 train(self, ...) 替换为规范的“仅推理并写结果”版本。


In [101]:
%cd /content/SORBET_ISWC23
from pathlib import Path
import re

tp = Path("src/train_pipeline.py")
code = tp.read_text(encoding="utf-8", errors="ignore")

# 在函数开头插入：writer 为空则直接返回（幂等，重复执行也不会出问题）
guard_pat = r'(def\s+_write_onto_embeddings_tensorboard\([^\)]*\):\s*\n)'
guard_ins = (
    r'\1'
    r'        # Guard for inference-only runs: skip if no TensorBoard writer\n'
    r'        if getattr(self, "tensorboard_writer", None) is None:\n'
    r'            return\n'
)
new_code, n = re.subn(guard_pat, guard_ins, code, count=1)
if n == 0:
    print("⚠️ 没找到 _write_onto_embeddings_tensorboard()，请把 src/train_pipeline.py 贴我前 300 行。")
else:
    tp.write_text(new_code, encoding="utf-8")
    print("✅ 已为 _write_onto_embeddings_tensorboard() 加入安全保护")


/content/SORBET_ISWC23
✅ 已为 _write_onto_embeddings_tensorboard() 加入安全保护


In [102]:
%cd /content/SORBET_ISWC23
from pathlib import Path, re

tp = Path("src/train_pipeline.py")
code = tp.read_text(encoding="utf-8", errors="ignore")

# 把所有： self._write_onto_embeddings_tensorboard( ... )
# 改成：
# if getattr(self, "tensorboard_writer", None) is not None:
#     try:
#         self._write_onto_embeddings_tensorboard( ... )
#     except Exception:
#         pass
pattern = re.compile(r'(?m)^(?P<indent>\s*)self\._write_onto_embeddings_tensorboard\((?P<args>.*)\)\s*$')
def repl(m):
    ind = m.group('indent')
    args = m.group('args')
    return (
        f'{ind}if getattr(self, "tensorboard_writer", None) is not None:\n'
        f'{ind}    try:\n'
        f'{ind}        self._write_onto_embeddings_tensorboard({args})\n'
        f'{ind}    except Exception:\n'
        f'{ind}        pass'
    )
new_code, n = pattern.subn(repl, code)
tp.write_text(new_code, encoding="utf-8")
print(f"✅ 已为 _write_onto_embeddings_tensorboard 的 {n} 处调用加上判空保护")


/content/SORBET_ISWC23
✅ 已为 _write_onto_embeddings_tensorboard 的 1 处调用加上判空保护


In [103]:
%cd /content/SORBET_ISWC23
!python src/train.py
# 顶部看到 cuDNN/cuBLAS 的告警属于环境噪声，可以忽略。


/content/SORBET_ISWC23
2025-10-10 22:18:17.230569: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760134697.250677   15829 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760134697.257113   15829 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760134697.272498   15829 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760134697.272528   15829 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760134697.272532   15829 computation_placer.cc:177]

In [104]:
import glob, os
dst = "/content/drive/MyDrive/sorbet_data/result_alignments"
hits = sorted(glob.glob(f"{dst}/**/*", recursive=True))
print("\n—— 写入目录 ——")
print(dst)
print("找到结果文件数：", len(hits))
for h in hits:
    if h.endswith((".tsv",".rdf")):
        print(" -", os.path.relpath(h, dst))


—— 写入目录 ——
/content/drive/MyDrive/sorbet_data/result_alignments
找到结果文件数： 15
 - energy_pair/Sargon2saref4bldg_-1.rdf
 - energy_pair/Sargon2saref4bldg_-1.tsv
 - energy_pair/Sargon2saref4bldg_-1_20251010-145758.rdf
 - energy_pair/Sargon2saref4bldg_-1_20251010-145758.tsv
 - energy_pair/Sargon2saref4bldg_-1_20251010-150505.rdf
 - energy_pair/Sargon2saref4bldg_-1_20251010-150505.tsv
 - energy_pair/Sargon2saref4bldg_-1_20251010-150843.rdf
 - energy_pair/Sargon2saref4bldg_-1_20251010-150843.tsv
 - energy_pair/saref4bldg2Sargon_-1.rdf
 - energy_pair/saref4bldg2Sargon_-1.tsv
 - run_20251010_145629/energy_pair/Sargon2saref4bldg_-1.rdf
 - run_20251010_145629/energy_pair/Sargon2saref4bldg_-1.tsv
